In [69]:
import nltk
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import contractions

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score

import string

In [5]:
# df = pd.read_csv("data/date_popularity_processed.csv")
# df.head()

,_id,duration,recorded_date,popularity,likes,views
0,21,992,2006-02-23,2.885684,17000,589115
1,1,957,2006-02-25,2.995805,110000,3671801
2,7,1266,2006-02-24,2.987323,60000,2008487
3,47,1126,2005-07-14,2.969279,80000,2694257
4,55,1524,2006-02-26,2.858560,14000,489757


In [7]:
df_org = pd.read_csv("data/talks_info_noedits.csv")

C:\Users\apfle\AppData\Local\Temp\ipykernel_12320\1484942858.py:1: DtypeWarning: Columns (24,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,2

In [8]:
df = df_org[['_id', 'duration', 'likes', 'speakers', 'subtitle_languages', 'summary', 'title', 'transcript', 'views', 'recorded_date']]

In [9]:
df.head()

,_id,duration,likes,speakers,subtitle_languages,summary,title,transcript,views,recorded_date
0,21,992,17K,"[{""name"":""Mena Trott"",""occupation"":""Blogger; c...","[{""name"":""English"",""code"":""en""},{""name"":""Vietn...","The founding mother of the blog revolution, Mo...",Meet the founder of the blog revolution,"Over the past couple of days, as I've been pre...",589115,2/23/06
1,1,957,110K,"[{""name"":""Al Gore"",""occupation"":""Climate advoc...","[{""name"":""Greek"",""code"":""el""},{""name"":""Gujarat...",With the same humor and humanity he exuded in ...,Averting the climate crisis,"Thank you so much, Chris. And it's truly a gre...",3671801,2/25/06
2,7,1266,60K,"[{""name"":""David Pogue"",""occupation"":""Technolog...","[{""name"":""Greek"",""code"":""el""},{""name"":""English...",New York Times columnist David Pogue takes aim...,Simplicity sells,"(Music: ""The Sound of Silence,"" Simon &amp; Ga...",2008487,2/24/06
3,47,1126,80K,"[{""name"":""David Deutsch"",""occupation"":""Physici...","[{""name"":""Greek"",""code"":""el""},{""name"":""English...",Legendary scientist David Deutsch puts theoret...,Chemical scum that dream of distant quasars,We've been told to go out on a limb and say so...,2694257,7/14/05
4,55,1524,14K,"[{""name"":""Jehane Noujaim"",""occupation"":""Filmma...","[{""name"":""English"",""code"":""en""},{""name"":""Vietn...",Jehane Noujaim unveils her 2006 TED Prize wish...,My wish: A global day of film,I can't help but this wish: to think about whe...,489757,2/26/06


In [11]:
print(df.head(1)["transcript"])

0    Over the past couple of days, as I've been pre...
Name: transcript, dtype: object


In [29]:
df["transcript"] = df.transcript.astype("string")

C:\Users\apfle\AppData\Local\Temp\ipykernel_12320\2823593348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["transcript"] = df.transcript.astype("string")


In [30]:
df.dtypes

_id                   object
duration              object
likes                 object
speakers              object
subtitle_languages    object
summary               object
title                 object
transcript            string
views                 object
recorded_date         object
dtype: object

In [23]:
df2 = df.copy()

In [31]:
df2["transcript"] = df.loc[:,"transcript"].astype("string")

In [35]:
transcript1 = df2[df2["_id"]=="21"]["transcript"]

In [38]:
df2[df2["_id"]=="21"]["title"]

0    Meet the founder of the blog revolution
Name: title, dtype: object

In [51]:
id = 0
for word in transcript1[0]:
    if id == 0:
        print(word)
#     print(word)
    id += 1

O


In [50]:
transcript1[0]

'Over the past couple of days, as I\'ve been preparing for my speech, I\'ve become more and more nervous about what I\'m going to say and about being on the same stage as all these fascinating people. Being on the same stage as Al Gore, who was the first person I ever voted for. And -- (Laughter) So I was getting pretty nervous and, you know, I didn\'t know that Chris sits on the stage, and that\'s more nerve-racking. But then I started thinking about my family. I started thinking about my father and my grandfather and my great-grandfather, and I realized that I had all of these Teds going through my bloodstream -- (Laughter) that I had to consider this "my element." So, who am I? Chris kind of mentioned I started a company with my husband. We have about 125 people internationally. If you looked in the book, you saw this ... (Laughter) which I really was appalled by. (Laughter) And because I wanted to impress you all with slides, since I saw the great presentations yesterday with graph

In [ ]:
# things to remove:
#     get the lemmatized version?
#     non-end of sentence punctuation
#     get rid of (Laughter), (Applause)
#     optional get rid of stop words

In [52]:
word_tokenize(transcript1[0])

['Over',
 'the',
 'past',
 'couple',
 'of',
 'days',
 ',',
 'as',
 'I',
 "'ve",
 'been',
 'preparing',
 'for',
 'my',
 'speech',
 ',',
 'I',
 "'ve",
 'become',
 'more',
 'and',
 'more',
 'nervous',
 'about',
 'what',
 'I',
 "'m",
 'going',
 'to',
 'say',
 'and',
 'about',
 'being',
 'on',
 'the',
 'same',
 'stage',
 'as',
 'all',
 'these',
 'fascinating',
 'people',
 '.',
 'Being',
 'on',
 'the',
 'same',
 'stage',
 'as',
 'Al',
 'Gore',
 ',',
 'who',
 'was',
 'the',
 'first',
 'person',
 'I',
 'ever',
 'voted',
 'for',
 '.',
 'And',
 '--',
 '(',
 'Laughter',
 ')',
 'So',
 'I',
 'was',
 'getting',
 'pretty',
 'nervous',
 'and',
 ',',
 'you',
 'know',
 ',',
 'I',
 'did',
 "n't",
 'know',
 'that',
 'Chris',
 'sits',
 'on',
 'the',
 'stage',
 ',',
 'and',
 'that',
 "'s",
 'more',
 'nerve-racking',
 '.',
 'But',
 'then',
 'I',
 'started',
 'thinking',
 'about',
 'my',
 'family',
 '.',
 'I',
 'started',
 'thinking',
 'about',
 'my',
 'father',
 'and',
 'my',
 'grandfather',
 'and',
 'my',
 

In [68]:
!pip install contractions

   --------------------------------------- 289.9/289.9 kB 18.6 MB/s eta 0:00:00


In [73]:
stopws = stopwords.words("english")
def process_text(conversation):
    
    contractions_stuff = [contractions.fix(word) for word in conversation.split()]
    conversation = ' '.join(contractions_stuff)
    
    conversation = conversation.lower()
    
    convo = "".join([char for char in conversation if char not in string.punctuation])
    
    words_tok = word_tokenize(convo)
    
    removed = [word for word in words_tok if word not in stopws]

    # lemmatization
    lemm = WordNetLemmatizer()
    lemmed = [lemm.lemmatize(word) for word in removed]
    
#     stemm = PorterStemmer()
#     stemmed = [stemm.stem(word) for word in removed]
    
    # put string together
    final_text = " ".join(lemmed)

    return lemmed

In [70]:
conversation = transcript1[0]
contractions_stuff = [contractions.fix(word) for word in conversation.split()]
expanded_text = ' '.join(contractions_stuff)
expanded_text

'Over the past couple of days, as I have been preparing for my speech, I have become more and more nervous about what I am going to say and about being on the same stage as all these fascinating people. Being on the same stage as Al Gore, who was the first person I ever voted for. And -- (Laughter) So I was getting pretty nervous and, you know, I did not know that Chris sits on the stage, and that is more nerve-racking. But then I started thinking about my family. I started thinking about my father and my grandfather and my great-grandfather, and I realized that I had all of these Teds going through my bloodstream -- (Laughter) that I had to consider this "my element." So, who am I? Chris kind of mentioned I started a company with my husband. We have about 125 people internationally. If you looked in the book, you saw this ... (Laughter) which I really was appalled by. (Laughter) And because I wanted to impress you all with slides, since I saw the great presentations yesterday with gra

In [72]:
process_text(transcript1[0])

'past couple day preparing speech become nervous going say stage fascinating people stage al gore first person ever voted laughter getting pretty nervous know know chris sits stage nerveracking started thinking family started thinking father grandfather greatgrandfather realized ted going bloodstream laughter consider element chris kind mentioned started company husband 125 people internationally looked book saw laughter really appalled laughter wanted impress slide since saw great presentation yesterday graph made graph move talk makeup laughter besides freakish thing science slide math science genetics grandmother get mouth laughter blogger probably lot mean different thing may heard kryptonite lock brouhaha blogger talked hack break kryptonite lock using ballpoint pen spread kryptonite adjust lock address avoid many customer concern may heard rathergate basically result blogger realizing th 111 typeset old typewriter word blogger exposed worked hard expose know blog scary see see su

In [78]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(process_text(conversation))
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = tfidf_matrix.toarray()

# 5. Evaluate the most important words per film
important_words_per_film = []
# for i, score in enumerate(tfidf_scores):
# Sort words by TF-IDF scores and select the top 30
important_words = [word for word, score in
                   sorted(zip(feature_names, score), key=lambda x: x[1], reverse=True)[:30]]
important_words_str = ", ".join(important_words)
important_words_per_film.append(important_words_str)
#     film_name = movie_transcripts[i]["title"]
#     important_words_per_film.append({"film": film_name, "words": important_words_str})

important_words_per_film

['applause, 100, 11, 111, 125, 17, 1910, 20, 2001, 2005, 23, 25, 28, 29, 300, 39, 55, 76, 77, 96, ability, able, access, account, action, actually, ad, add, address, adjust']

In [79]:
tfidf_scores

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
feature_names

array(['100', '11', '111', '125', '17', '1910', '20', '2001', '2005',
       '23', '25', '28', '29', '300', '39', '55', '76', '77', '96',
       'ability', 'able', 'access', 'account', 'action', 'actually', 'ad',
       'add', 'address', 'adjust', 'affect', 'age', 'aging', 'ago', 'al',
       'also', 'always', 'amazing', 'amount', 'another', 'anymore',
       'anything', 'anyways', 'apnea', 'appalled', 'applause', 'april',
       'archive', 'argentinean', 'art', 'asshole', 'attack', 'audience',
       'avoid', 'award', 'away', 'baby', 'back', 'bad', 'ballpoint',
       'banjo', 'bank', 'basically', 'bathroom', 'become', 'beer', 'ben',
       'besides', 'best', 'beta', 'better', 'bible', 'biblical', 'big',
       'billion', 'birth', 'bit', 'blog', 'blogger', 'blogging',
       'bloodstream', 'book', 'break', 'breathing', 'brouhaha', 'bunk',
       'buy', 'call', 'came', 'camp', 'cancer', 'capture', 'caring',
       'census', 'change', 'changing', 'child', 'china', 'choked',
       'chri